Nuovi tentativi di merge a partire da soli ds ANAC

Script Borruso
- Scarica tutti i csv di delta aggiornamenti di CIG bandi gara (ANAC, 23) zippati, li decomprimere e li converte in gzip così sono accessibili in stream;
- Filtra da tutti i file scaricati soltanto quelli con flag PNRR uguale a 1;
- Per cig duplicati, prende il più recente;
- Fa il merge di tutti i file in uno solo

P03 : dataset CIG - MISURE PREMIALI

PNEW : dataset CIG - anagrafica CIG [per CIG PNRR, 2023]

P04 : dataset CIG - quote/deroghe

COMUNI : Istat

In [1]:
import pandas as pd
import numpy as np 

In [2]:
# datasets
pnew = pd.read_csv("./datasets/pnrr.csv",delimiter=',',encoding = "UTF-8")
p03 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="misurepremiali-pnrrpnc_csv")
p04 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="indicatori-pnrrpnc_csv")

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_17648\3709095954.py:2: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  pnew = pd.read_csv("./datasets/pnrr.csv",delimiter=',',encoding = "UTF-8")


In [4]:
p06 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="PNRR_Gare-Dati_Validati")

In [3]:
comuni = pd.read_csv("./datasets/Elenco-comuni-italiani.csv",delimiter=';',encoding = "ISO-8859-1")

In [3]:
# preprocessing
pnew = pnew.rename(columns={'cig':'CIG'})
p03 = p03.rename(columns={'cig':'CIG'})
p04 = p04.rename(columns={'cig':'CIG'})

In [12]:
p04['flag_quote'].unique()

array(['N', 'Q', 'S', nan], dtype=object)

In [5]:
pnew['CIG'].nunique()

177312

In [6]:
entries = pnew['oggetto_gara']
pnew_missioni = pnew[pd.Series(entries).str.contains(r'MISSIONE')]

In [16]:
pnew_missioni.shape

(15333, 62)

Aggiunta colonne in p03 e pnew di categoria/importo/regione

- categoria

In [7]:
def conditions(s):
    if s['cod_mis_premiale'] in [1,2,9]:
        return "GENERALE"
    elif s['cod_mis_premiale'] in [3,8,12]:
        return "DISABILI"
    elif s['cod_mis_premiale'] in [4,6,10]:
        return "GENERE"
    elif s['cod_mis_premiale'] in [5,7,11]:
        return "ETÀ"
    else:
        return 0
    
p03['CATEGORIA'] = p03.apply(conditions,axis=1)

- importo

In [7]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [8]:
min_importo = pnew['importo_complessivo_gara'].min()
max_importo = pnew["importo_complessivo_gara"].max()

In [9]:
def map_importo(importo):
    if importo <= 100000:
        return 'BASSA'
    elif importo <= 1000000:
        return 'MEDIA'
    else:
        return 'ALTA'
    
pnew['CLASSE_IMPORTO'] = pnew['importo_complessivo_gara'].apply(map_importo)

In [9]:
pnew['oggetto_gara'].unique()

array(['INTEGRAZIONE ED EVOLUZIONE DEL MODULO GPP CON MPP E MOSS E CANONE MANUTENZIONE INTEGRAZIONE',
       'MISURA 1.4.5.SPID PIATTAFORMA NOTIFICHE DIGITALI PNRR M1C1',
       "MISURA 1.4.1 'ESPERIENZA DEL CITTADINO NEI SERVIZI PUBBLICI' ",
       ...,
       'AFFIDAMENTO LAVORI DI: RIPRISTINO MANTI DI COPERTURA DI EDIFICI SCOLASTICI',
       "AFFIDAMENTO DIRETTO, AI SENSI DELL'ART. 1, COMMA 2, LETT. A) DEL D.LGS. N. 76/2020, CONVERTITO CON MODIFICAZIONI, DALLA L. N.120/2020, PER L'ATTRIBUZIONE DELL'INCARICO DI REDAZIONE DELLA PROGETTAZIONE ESECUTIVA, COORDINAMENTO DELLA SICUREZZA IN FASE DI PROGETTAZIONE ED ESECUZIONE E DIREZIONE LAVORI DELL INTERVENTO PER LA FORNITURA E INSTALLAZIONE DI UNA SECONDA TAC PRESSO IL P.O. 'MAGGIORE - NINO BAGLIERI' DI MODICA",
       'PNRR LABS RIFACIMENTO IMPIANTO ELETTRICO SECONDO NORMA DI LEGGE'],
      dtype=object)

- regione

In [11]:
pnew['provincia'].isna().sum() # 19494 CIG senza luogo associato

19494

In [12]:
# file istat
prov_reg = comuni[['Denominazione Regione','Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)']]

In [13]:
prov_reg = prov_reg.rename(columns={'Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)':'Denominazione sovracomunale','Denominazione Regione':'Regione'})

In [14]:
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Valle d'Aosta/Vallée d'Aoste", value='Aosta', regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Bolzano/Bozen", value='Bolzano',regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].str.upper()
prov_reg['Denominazione sovracomunale'].unique()

array(['TORINO', 'VERCELLI', 'NOVARA', 'CUNEO', 'ASTI', 'ALESSANDRIA',
       'BIELLA', 'VERBANO-CUSIO-OSSOLA', 'AOSTA', 'VARESE', 'COMO',
       'SONDRIO', 'MILANO', 'BERGAMO', 'BRESCIA', 'PAVIA', 'CREMONA',
       'MANTOVA', 'LECCO', 'LODI', 'MONZA E DELLA BRIANZA', 'BOLZANO',
       'TRENTO', 'VERONA', 'VICENZA', 'BELLUNO', 'TREVISO', 'VENEZIA',
       'PADOVA', 'ROVIGO', 'UDINE', 'GORIZIA', 'TRIESTE', 'PORDENONE',
       'IMPERIA', 'SAVONA', 'GENOVA', 'LA SPEZIA', 'PIACENZA', 'PARMA',
       "REGGIO NELL'EMILIA", 'MODENA', 'BOLOGNA', 'FERRARA', 'RAVENNA',
       'FORLÌ-CESENA', 'RIMINI', 'MASSA-CARRARA', 'LUCCA', 'PISTOIA',
       'FIRENZE', 'LIVORNO', 'PISA', 'AREZZO', 'SIENA', 'GROSSETO',
       'PRATO', 'PERUGIA', 'TERNI', 'PESARO E URBINO', 'ANCONA',
       'MACERATA', 'ASCOLI PICENO', 'FERMO', 'VITERBO', 'RIETI', 'ROMA',
       'LATINA', 'FROSINONE', "L'AQUILA", 'TERAMO', 'PESCARA', 'CHIETI',
       'CAMPOBASSO', 'ISERNIA', 'CASERTA', 'BENEVENTO', 'NAPOLI',
       'AVELLINO', 

In [15]:
prov_reg=prov_reg.drop_duplicates()

In [16]:
pnew=pnew.merge(prov_reg,left_on='provincia', right_on='Denominazione sovracomunale',how='left')

In [17]:
pnew

,CIG,cig_accordo_quadro,numero_gara,oggetto_gara,importo_complessivo_gara,n_lotti_componenti,oggetto_lotto,importo_lotto,oggetto_principale_contratto,stato,...,IPOTESI_COLLEGAMENTO,CIG_COLLEGAMENTO,COD_ESITO,ESITO,DATA_COMUNICAZIONE_ESITO,FLAG_PNRR_PNC,file,CLASSE_IMPORTO,Regione,Denominazione sovracomunale
0,94965511CD,NaN,8803480,INTEGRAZIONE ED EVOLUZIONE DEL MODULO GPP CON ...,135244.23,1.00,INTEGRAZIONE ED EVOLUZIONE DEL MODULO GPP CON ...,135244.23,SERVIZI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,MEDIA,NaN,NaN
1,9719455BFD,NaN,9003805,MISURA 1.4.5.SPID PIATTAFORMA NOTIFICHE DIGITA...,18972.95,1.00,MISURA 1.4.5.SPID PIATTAFORMA NOTIFICHE DIGITA...,18972.95,SERVIZI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,BASSA,Sicilia,MESSINA
2,9576444B94,NaN,8878071,MISURA 1.4.1 'ESPERIENZA DEL CITTADINO NEI SER...,243287.16,1.00,MISURA 1.4.1 'ESPERIENZA DEL CITTADINO NEI SER...,243287.16,SERVIZI,ATTIVO,...,NaN,NaN,7.00,ANNULLATA/REVOCATA DOPO L'APERTURA DELLE BUSTE...,2023-04-04,1,20231101-cig_csv.gz,MEDIA,Sicilia,MESSINA
3,9583207896,NaN,8884263,PROCEDURA APERTA DA ESPLETARSI IN MANIERA TELE...,287648.76,1.00,PROCEDURA APERTA DA ESPLETARSI IN MANIERA TELE...,287648.76,SERVIZI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,MEDIA,Puglia,TARANTO
4,9469637FA6,NaN,8779273,"PON LABORATORI GREEN, SOSTENIBILI E INNOVATIVI...",4500.00,1.00,"PON LABORATORI GREEN, SOSTENIBILI E INNOVATIVI...",4500.00,FORNITURE,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,BASSA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177307,94455244FC,NaN,8757718,ESPERIENZA DEL CITTADINO NEI SERVIZI PUBBLICI,79922.00,2.00,ESPERIENZA DEL CITTADINO NEI SERVIZI PUBBLICI ...,30922.00,SERVIZI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,BASSA,Veneto,VERONA
177308,A00B0BA7B2,NaN,9301932,ABILITAZIONE AL CLOUD PER LE PA LOCALI' SCUOLE...,3626.23,1.00,ABILITAZIONE AL CLOUD PER LE PA LOCALI' SCUOLE...,3626.23,SERVIZI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,BASSA,Toscana,LUCCA
177309,A0173FA7CC,NaN,9345564,AFFIDAMENTO LAVORI DI: RIPRISTINO MANTI DI COP...,130366.07,1.00,AFFIDAMENTO LAVORI DI: 'RIPRISTINO MANTI DI CO...,130366.07,LAVORI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,MEDIA,Campania,NAPOLI
177310,9998983DCA,NaN,9258977,"AFFIDAMENTO DIRETTO, AI SENSI DELL'ART. 1, COM...",21034.21,1.00,"AFFIDAMENTO DIRETTO, AI SENSI DELL'ART. 1, COM...",21034.21,SERVIZI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,1,20231101-cig_csv.gz,BASSA,Sicilia,RAGUSA


- missione

In [ ]:
"""rdc = testo.lower().find("reddito di cittadinanza")
        pre = testo[:rdc]
        end = testo.find(". ",rdc)
        paragraph = testo[pre.rfind(". ")+2: end+2]"""

Joins

In [18]:
# inner join p3 e pNEW
innerNEW_3 = p03.merge(pnew, on='CIG',how='inner')

In [19]:
# inner join p4 e pNEW
innerNEW_4 = p04.merge(pnew, on='CIG',how='inner')

In [24]:
p04['CIG'].nunique()
#p03['CIG'].nunique()

178550

In [160]:
# left join pNEW e p3
leftnew_3 = pnew.merge(p03, on='CIG',how='left')

In [161]:
leftnew_3=leftnew_3.drop_duplicates()
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG',how='left')

In [166]:
leftnew_34=leftnew_34.drop_duplicates()

In [167]:
leftnew_34['CIG'].nunique()

177312

In [168]:
leftnew_34.to_csv("CIG_prem_deroghe.csv", index = False, sep=';')
pd.read_csv("CIG_prem_deroghe.csv", sep=';')

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_17720\2854529410.py:2: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("CIG_prem_deroghe.csv", sep=';')


,CIG,cig_accordo_quadro,numero_gara,oggetto_gara,importo_complessivo_gara,n_lotti_componenti,oggetto_lotto,importo_lotto,oggetto_principale_contratto,stato,...,Regione,Denominazione sovracomunale,cod_mis_premiale,misura_premiale,CATEGORIA,flag_quote,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga
0,94965511CD,NaN,8803480,INTEGRAZIONE ED EVOLUZIONE DEL MODULO GPP CON ...,135244.23,1.00,INTEGRAZIONE ED EVOLUZIONE DEL MODULO GPP CON ...,135244.23,SERVIZI,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN
1,9719455BFD,NaN,9003805,MISURA 1.4.5.SPID PIATTAFORMA NOTIFICHE DIGITA...,18972.95,1.00,MISURA 1.4.5.SPID PIATTAFORMA NOTIFICHE DIGITA...,18972.95,SERVIZI,ATTIVO,...,Sicilia,MESSINA,NaN,NaN,NaN,N,NaN,NaN,1.00,IMPORTO RIDOTTO DEL CONTRATTO
2,9576444B94,NaN,8878071,MISURA 1.4.1 'ESPERIENZA DEL CITTADINO NEI SER...,243287.16,1.00,MISURA 1.4.1 'ESPERIENZA DEL CITTADINO NEI SER...,243287.16,SERVIZI,ATTIVO,...,Sicilia,MESSINA,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN
3,9583207896,NaN,8884263,PROCEDURA APERTA DA ESPLETARSI IN MANIERA TELE...,287648.76,1.00,PROCEDURA APERTA DA ESPLETARSI IN MANIERA TELE...,287648.76,SERVIZI,ATTIVO,...,Puglia,TARANTO,NaN,NaN,NaN,N,NaN,NaN,8.00,ALTRO
4,9469637FA6,NaN,8779273,"PON LABORATORI GREEN, SOSTENIBILI E INNOVATIVI...",4500.00,1.00,"PON LABORATORI GREEN, SOSTENIBILI E INNOVATIVI...",4500.00,FORNITURE,ATTIVO,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,1.00,IMPORTO RIDOTTO DEL CONTRATTO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205950,A00B0BA7B2,NaN,9301932,ABILITAZIONE AL CLOUD PER LE PA LOCALI' SCUOLE...,3626.23,1.00,ABILITAZIONE AL CLOUD PER LE PA LOCALI' SCUOLE...,3626.23,SERVIZI,ATTIVO,...,Toscana,LUCCA,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN
205951,A0173FA7CC,NaN,9345564,AFFIDAMENTO LAVORI DI: RIPRISTINO MANTI DI COP...,130366.07,1.00,AFFIDAMENTO LAVORI DI: 'RIPRISTINO MANTI DI CO...,130366.07,LAVORI,ATTIVO,...,Campania,NAPOLI,NaN,NaN,NaN,N,NaN,NaN,1.00,IMPORTO RIDOTTO DEL CONTRATTO
205952,9998983DCA,NaN,9258977,"AFFIDAMENTO DIRETTO, AI SENSI DELL'ART. 1, COM...",21034.21,1.00,"AFFIDAMENTO DIRETTO, AI SENSI DELL'ART. 1, COM...",21034.21,SERVIZI,ATTIVO,...,Sicilia,RAGUSA,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN
205953,998232070E,NaN,9243076,PNRR LABS RIFACIMENTO IMPIANTO ELETTRICO SECON...,8176.00,1.00,PNRR LABS RIFACIMENTO IMPIANTO ELETTRICO SECON...,8176.00,SERVIZI,ATTIVO,...,Marche,PESARO E URBINO,NaN,NaN,NaN,N,NaN,NaN,1.00,IMPORTO RIDOTTO DEL CONTRATTO


In [18]:
print('CIG unici in P03 [MISURE PREMIALI]: ',p03['CIG'].nunique())
print('CIG unici in P04 [DEROGHE]: ',p04['CIG'].nunique())
print('CIG unici in PNEW [CIG TOT 2023]: ', pnew['CIG'].nunique())
print('CIG unici in P03 presenti anche in pNEW [CIG TOT 2023]',innerNEW_3['CIG'].nunique())
print('CIG unici in P04 presenti anche in pNEW [CIG TOT 2023]',innerNEW_4['CIG'].nunique())

CIG unici in P03 [MISURE PREMIALI]:  11402
CIG unici in P04 [DEROGHE]:  178550
CIG unici in PNEW [CIG TOT 2023]:  177312
CIG unici in P03 presenti anche in pNEW [CIG TOT 2023] 11265
CIG unici in P04 presenti anche in pNEW [CIG TOT 2023] 177297


In [19]:
count_CIG_p3 = p03['CIG'].nunique()
count_CIG_p4 = p04['CIG'].nunique()
count_CIG_new3 = innerNEW_3['CIG'].nunique()
count_CIG_new4 = innerNEW_4['CIG'].nunique()

In [20]:
print(np.round(count_CIG_new3/count_CIG_p3*100, 2),'%')
print(np.round(count_CIG_new4/count_CIG_p4*100, 2),'%') # per p04 -> 99.3%

98.8 %
99.3 %


98.8% dei CIG in p03 hanno un match nel dataset new

99.3 % dei CIG in p04 hanno un match nel dataset new

In [21]:
print(np.round((11265/177312)*100,2),'%')
print(np.round((11402/177312)*100,2),'%')

6.35 %
6.43 %


Sul totale dei CIG del 2023, 6.35% dei CIG del 2023 hanno premialità

[6.43 % se si considerano 1.2% di CIG con premialità - 137 su 11402 - senza match in pNEW]

In [25]:
print(np.round((count_CIG_new4/177312)*100,2),'%')

99.99 %


Sul totale dei CIG del 2023, 99.99 % dei CIG del 2023 hanno informazioni sulle quote/deroghe.

Info presenti: CIG, importo, provincia/luogo istat, data

In [22]:
innerNEW_3['sezione_regionale'].isna().sum()

0

Confronto col dataset p06 [fino ad ora]

In [12]:
# inner join p3 e p6
temp3_6 = p03.merge(p06, on='CIG',how='inner')

In [13]:
temp4_6 = p04.merge(p06, on='CIG',how='inner')

In [14]:
print('CIG unici in P06: ', p06['CIG'].nunique())
print('CIG unici in P03 [MISURE PREMIALI]: ',p03['CIG'].nunique())
print('CIG unici in P04 [DEROGHE]: ',p04['CIG'].nunique())
print('CIG unici in P03 presenti anche in p06:',temp3_6['CIG'].nunique())
print('CIG unici in P04 presenti anche in p06:',temp4_6['CIG'].nunique())

CIG unici in P06:  56984
CIG unici in P03 [MISURE PREMIALI]:  11402
CIG unici in P04 [DEROGHE]:  178550
CIG unici in P03 presenti anche in p06: 2215
CIG unici in P04 presenti anche in p06: 19727


In [15]:
print(np.round((2215/11402)*100,2),'%')
print(np.round((19727/178550)*100,2),'%')

19.43 %
11.05 %


19.43 % dei CIG in p03 hanno un match nel dataset P06

11.05 % dei CIG in p04 hanno un match nel dataset P06

In [69]:
p06.columns

Index(['Codice Univoco Submisura', 'Descrizione Submisura',
       'Descrizione Procedura di Aggiudicazione', 'CUP',
       'Codice Locale Progetto', 'CIG', 'CIG Accordo Quadro',
       'Codice Procedura Utente', 'Modalit� di Realizzazione',
       'Codice Interno PDA', 'Oggetto Principale del Contratto',
       'Oggetto Gara', 'Data Pubblicazione del CIG',
       'Codice Motivo Assenza CIG', 'Descrizione Motivo Assenza CIG',
       'Importo Complessivo Gara', 'Importo Aggiudicazione',
       'Data Aggiudicazione Definitiva', 'Data di Aggiornamento'],
      dtype='object')

In [10]:
p01 = pd.read_csv("./datasets/cup_csv.csv",delimiter=';',encoding = "ISO-8859-1")

In [11]:
pnew_cup = p01.merge(pnew, on='CIG', how='inner')

In [13]:
pnew['CIG'].nunique()

177312

In [14]:
pnew_cup['CIG'].nunique()

51313